In [1]:
# загрузим необходимые библиотеки
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import random
import string

# Определяем устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Функция для шифрования текста
def caesar_cipher(text, shift):
    result = []
    for char in text:
        if char in CHAR_TO_INDEX:
            idx = (CHAR_TO_INDEX[char] + shift) % VOCAB_SIZE
            result.append(INDEX_TO_CHAR[idx])
        else:
            result.append(char)  # если символ не в алфавите, оставляем как есть
    return "".join(result)

In [ ]:
# Генерация данных
def generate_data(num_samples):
    data = []
    for _ in range(num_samples):
        # Генерируем случайную фразу длиной от 10 до MAX_LEN символов
        length = random.randint(10, MAX_LEN)
        text = "".join(random.choices(ALPHABET, k=length))
        shift = random.randint(1, 10)  # случайный сдвиг от 1 до 10
        encrypted_text = caesar_cipher(text, shift)  # шифруем текст
        data.append((encrypted_text, text, shift))  # сохраняем зашифрованный текст, исходный текст и сдвиг
    return data

#### 2. Построим нейронную сеть

1. Разбьем данные на токены (у нас символы)
2. Закодируем числами
3. Превратим в эмбеддинги

In [ ]:
# Параметры
ALPHABET = string.ascii_lowercase + " "  # алфавит (буквы + пробел)
CHAR_TO_INDEX = {char: idx for idx, char in enumerate(ALPHABET)}  # символ → индекс
INDEX_TO_CHAR = {idx: char for char, idx in CHAR_TO_INDEX.items()}  # индекс → символ
VOCAB_SIZE = len(ALPHABET)  # размер словаря
MAX_LEN = 50  # максимальная длина фразы

In [ ]:
# Преобразование текста в тензор
def text_to_tensor(text, max_len):
    tensor = torch.zeros(max_len, dtype=torch.long).to(device)  # перемещаем на устройство
    for i, char in enumerate(text[:max_len]):
        tensor[i] = CHAR_TO_INDEX.get(char, CHAR_TO_INDEX[" "])  # неизвестные символы → пробел
    return tensor

In [ ]:
HIDDEN_SIZE = 256  # увеличенный размер скрытого состояния
EPOCHS = 30  # увеличенное количество эпох
BATCH_SIZE = 128  # увеличенный размер батча
LEARNING_RATE = 0.001  # уменьшенный learning rate
DROPOUT = 0.2  # добавлен Dropout

## Реализация сети с RNN
3 слоя:
1. Embeding (30)
2. RNN (hidden_dim=128)
3. Полносвязный слой для предсказания буквы (28, то есть размер словаря)

In [ ]:
# Нейронная сеть с учетом сдвига
class CaesarDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout):
        super(CaesarDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.shift_embedding = nn.Embedding(11, hidden_size)  # сдвиги от 0 до 10
        self.rnn = nn.GRU(hidden_size * 2, hidden_size, batch_first=True, num_layers=2, dropout=dropout)  # вход: символы + сдвиг
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, shift):
        char_emb = self.embedding(x)  # эмбеддинги символов
        shift_emb = self.shift_embedding(shift).unsqueeze(1).expand(-1, x.size(1), -1)  # эмбеддинги сдвига
        combined = torch.cat((char_emb, shift_emb), dim=2)  # объединяем символы и сдвиг
        x, _ = self.rnn(combined)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
# Подготовка данных
data = generate_data(20000)  # генерируем 20,000 примеров
encrypted_texts, original_texts, shifts = zip(*data)

In [ ]:
# Преобразуем тексты в тензоры
X = torch.stack([text_to_tensor(text, MAX_LEN) for text in encrypted_texts]).to(device)  # перемещаем на устройство
Y = torch.stack([text_to_tensor(text, MAX_LEN) for text in original_texts]).to(device)  # перемещаем на устройство
shifts = torch.tensor(shifts, dtype=torch.long).to(device)  # преобразуем сдвиги в тензор

In [ ]:
# Создаем модель, функцию потерь и оптимизатор
model = CaesarDecoder(VOCAB_SIZE, HIDDEN_SIZE, DROPOUT).to(device)  # перемещаем модель на устройство
criterion = nn.CrossEntropyLoss().to(device)  # перемещаем функцию потерь на устройство
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)  # используем Adam
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # добавляем scheduler


Обучение:

In [ ]:
# Обучение модели
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_correct = 0  # для подсчета правильных предсказаний
    total_samples = 0  # для подсчета общего числа символов

    for i in range(0, len(X), BATCH_SIZE):
        # Берем батч
        X_batch = X[i:i + BATCH_SIZE]
        Y_batch = Y[i:i + BATCH_SIZE]
        shift_batch = shifts[i:i + BATCH_SIZE]  # сдвиги для батча

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(X_batch, shift_batch)  # передаем сдвиг в модель
        outputs = outputs.view(-1, VOCAB_SIZE)
        Y_batch = Y_batch.view(-1)

        # Вычисляем потери
        loss = criterion(outputs, Y_batch)
        total_loss += loss.item()

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()

        # Вычисляем accuracy
        _, predicted = torch.max(outputs, dim=1)  # получаем предсказанные символы
        total_correct += (predicted == Y_batch).sum().item()  # считаем правильные предсказания
        total_samples += Y_batch.size(0)  # общее количество символов в батче

    # Обновляем learning rate
    scheduler.step()

    # Выводим статистику
    accuracy = total_correct / total_samples  # вычисляем accuracy
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / (len(X) // BATCH_SIZE):.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
# Проверка качества
def decode_text(model, encrypted_text, shift):
    model.eval()
    with torch.no_grad():
        # Ограничиваем длину входного текста
        input_length = len(encrypted_text)
        tensor = text_to_tensor(encrypted_text, input_length).unsqueeze(0).to(device)  # перемещаем на устройство
        shift_tensor = torch.tensor([shift], dtype=torch.long).to(device)  # преобразуем сдвиг в тензор
        output = model(tensor, shift_tensor)
        _, indices = torch.max(output, dim=2)
        decoded_text = "".join([INDEX_TO_CHAR[idx.item()] for idx in indices[0][:input_length]])  # обрезаем до длины исходного текста
    return decoded_text

In [ ]:
# Пример использования
texts = [
    "hello world", "neural network", "caesar cipher", "deep learning",
    "machine learning", "artificial intelligence", "data science",
    "python programming", "pytorch framework", "encryption algorithms"
]

print("Testing the model on multiple examples:")
for text in texts:
    shift = random.randint(1, 10)  # случайный сдвиг от 1 до 10
    encrypted_text = caesar_cipher(text, shift)
    decoded_text = decode_text(model, encrypted_text, shift)
    print(f"Original: {text}")
    print(f"Shift: {shift}")
    print(f"Encrypted: {encrypted_text}")
    print(f"Decoded: {decoded_text}")
    print("-" * 40)